<a href="https://colab.research.google.com/github/alexcu/fastai-primer/blob/main/fastai-primer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fast AI Primer 🤖

_Learn how to train a semi-decent machine learning model in 5 steps using [FastAI](https://fast.ai)!_

<small>**Disclaimer:** This is a quick-n-dirty primer and skips a lot of important details. If you are interested in more info, check out FastAI's [tutorial](https://docs.fast.ai/tutorial.vision.html) or [free course](https://course.fast.ai).</small>

## 0️⃣: Prerequsites

Make sure you give your notebook a GPU to work with!

<img src="https://i.imgur.com/GX9QaIN.png" width="600">

Then run the following code by pressing <strong><kbd>⌘</kbd>+<kbd>Enter</kbd></strong> (or hover over the cell below and click the ▶ button) to download and set up FastAI.

Authorise your application when prompted:

<img src="https://i.imgur.com/5eqPT7L.png" width="600">


In [ ]:
!pip install -Uqq fastbook
!pip install --upgrade git+https://github.com/fastai/fastai.git
!apt-get -y install jq
% matplotlib inline
import fastbook
fastbook.setup_book()

## 1️⃣: Download some images from the [Creative Commons API](https://api.creativecommons.engineering/v1/#operation/image_search)

Replace `labels` on line 17 to train something different. <br><small>(If line numbers are off, you can enable them in Tools 👉 Settings 👉 Editor 👉 Show Line Numbers.)</small>

Here are some ideas you can try out:

Idea|Labels
---|---
🎨&nbsp;&nbsp;&nbsp;Art Movements|`["impressionism", "cubism art", "pop art"]`
🍦&nbsp;&nbsp;&nbsp;Ice Cream Flavours|`["chocolate ice cream", "vanilla ice cream", "mint ice cream"]`
🐕&nbsp;&nbsp;&nbsp;Dog Breeds|`["pug", "chiwawa", "shiba inu"]`
💐&nbsp;&nbsp;&nbsp;Flowers|`["sunflowers", "lavender", "roses"]`
🍺&nbsp;&nbsp;&nbsp;Beverages|`["beer", "whisky", "red wine"]`
🧀&nbsp;&nbsp;&nbsp;Cheeses|`["blue cheese", "mozzarella", "camembert"]`

<small>**NB:** Add `--show-progress` after `-q` on line 34 if the `wget` command is taking too to monitor progress...</small>

In [ ]:
from pathlib import Path

# Directory where we will download our all our images - /datasets/{label}
datasets_dir = Path("/datasets")

# We can run shell commands using ! and insert variables with $
# E.g., make sure our datasets_dir is empty
! rm -rf "$datasets_dir"

# Total number of images per label in our dataset (max is 500)
num_imgs_per_label = 200

# Quality of images (small/medium/large)
image_quality = "medium"

# Label(s) we want to search images for and train on
labels = ["impressionism", "cubism art", "pop art"]

for label in labels:
  # Create a directory where we'll download all the images for this label
  label_dataset_dir = datasets_dir / label
  ! mkdir -p "$label_dataset_dir"

  # Where we'll store a txt file of all URLs for us to download
  label_urls_txt = datasets_dir / f"{label}-imgs-urls.txt"
  
  # Request to Creative Commons API and download the results
  cc_img_api_url = (
      f"https://api.openverse.engineering/v1/images?"
      f"page_size={num_imgs_per_label}&"
      f"extension=jpg&"
      f"size={image_quality}&"
      f"q=" + label.replace(" ", "%20")
  )

  # Bearer Token for Creative Commons API (see https://bit.ly/32XpSav)
  bearer_token = 'DLBYIcfnKfolaXKcmMC8RIDCavc2hW'

  print(f"* Requesting {num_imgs_per_label} random '{label}' images:\n  {cc_img_api_url}")
  ! curl -Ls -H "Authorization: Bearer $bearer_token" "$cc_img_api_url"\
      |  jq -r '.results[].url' > "$label_urls_txt"
  ! echo "* Got $(wc -l < '$label_urls_txt') images of '$label'. Downloading..."
  ! wget -q -i "$label_urls_txt" -P "$label_dataset_dir"

print("Done!")

### What images did we download?

Re-run the cell below to shuffle through some of the images we downloaded.

FastAI comes with some neat helper functions (some which wrap [PyTorch](https://pytorch.org)). E.g., here we use:

* [**`get_image_files`**](https://docs.fast.ai/data.transforms.html#get_image_files) to return a list of image files in a directory
* [**`load_image`**](https://docs.fast.ai/vision.core.html#load_image) to load an image into memory
* [**`show_images`**](https://docs.fast.ai/torch_core.html#show_images) to show multiple images in a grid using [matplotlib](https://matplotlib.org)

<small>**NB:** Importing FastAI with a wildcard is considered 'safe' according to its authors...</small>

In [ ]:
from fastai.vision.all import *

for label in labels:
  images = get_image_files(datasets_dir, folders=label)
  sample = [load_image(image) for image in images.shuffle()[:5]]
  show_images(sample, nrows=1, ncols=5, imsize=3, suptitle=label)

## 2️⃣: Loading the images into memory

An important part of training a model is to see how well it handles previously-unseen data.

So, we can split the data we downloaded into two datasets:

  * a **training dataset**, to actually teach the model what labels to look for;
  * a **validation dataset**, to validate whether the model can accurately assess unseen images.

FastAI has a helper function, [**`ImageDataLoaders.from_folder`**](https://docs.fast.ai/vision.data.html#ImageDataLoaders.from_folder), to help us load  these two datasets into memory.

In [ ]:
# How much of our data we will reserve for training vs. validation
validation_dataset_pct = 0.35

# Resize all images consistently to 256x256 pixels
#  ↑ resolution = ↑ accuracy = ↑ computation power = ↑ training time
images_transformations = Resize(256)

print(f"* Loading images from {datasets_dir}:")
print(f"    We will use {len(labels)} labels: {labels}")
print(f"    We will reserve {1-validation_dataset_pct:.0%} of our data for training")

dataloader = ImageDataLoaders.from_folder(
  datasets_dir,                     # <- Where our data is stored (i.e., /datasets)
  vocab=labels,                     # <- What subdirectories we have (one for each label)
  valid_pct=validation_dataset_pct, # <- % of data we set aside for validation
  item_tfms=images_transformations  # <- Transformations to make to each image
)

for label in labels:
  num_train_imgs = len([path for path in dataloader.train_ds.items if path.parent.name == label])
  num_valid_imgs = len([path for path in dataloader.valid_ds.items if path.parent.name == label])
  print(f"* For '{label}...'")
  print(f"    We have {num_train_imgs} training images")
  print(f"    We have {num_valid_imgs} validation images")

## 3️⃣: Training the model

FastAI has [different **pre-trained models**](https://fastai1.fast.ai/vision.models.html#Computer-Vision-models-zoo) which we can piggy back off:

Architecture|First Appeared|Available Implementations In FastAI
---|---|---
[AlexNet](https://en.wikipedia.org/wiki/AlexNet)|[2012](https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf)|`alexnet`
[Resnet](https://en.wikipedia.org/wiki/Residual_neural_network)|[2015](https://arxiv.org/abs/1512.03385)|`resnet18,resnet34,resnet50,resnet101,resnet152`
[Squeezenet](https://en.wikipedia.org/wiki/SqueezeNet)|[2016](https://arxiv.org/abs/1602.07360)|`squeezenet1_0,squeezenet1_1`
[Densenet](https://paperswithcode.com/method/densenet)|[2016](https://arxiv.org/abs/1608.06993)|`densenet121,densenet169,densenet201,densenet161`

We'll select one of these ☝️ as our _base pretrained model_ and use FastAI's [**`cnn_learner`**](https://docs.fast.ai/vision.learner.html#cnn_learner) to set up a [Convolutional Neural Network](https://en.wikipedia.org/wiki/Convolutional_neural_network) for us to play with.

We'll _specialise_ the base model to our specific purpose by calling [**`fine_tune`**](https://docs.fast.ai/callback.schedule.html#Learner.fine_tune), which trains the model.

<details>
<summary>Why do we use a base model?</summary>

Here's a quote from [Neurohive](https://neurohive.io/en/popular-networks/alexnet-imagenet-classification-with-deep-convolutional-neural-networks/) about [AlexNet](https://en.wikipedia.org/wiki/AlexNet):

> AlexNet was trained for 6 days simultaneously on two Nvidia Geforce GTX 580 GPUs on ImageNet, a dataset of over 15 million labeled high-resolution images belonging to roughly 22,000 categories.

We use a base model so we don't have to do that ☝️.

Here is a visualisation and explanation of each of AlexNet's 5 layers, and what _features_ it learnt ([source](https://arxiv.org/pdf/1311.2901.pdf)). The model we train _piggy-backs_ off of what the base model already knows.

Layer|Visualisation of Weights Learned|Patches of example Training Data|Description
---|---|---|---
1|![](https://i.imgur.com/hDKs1b7.png)|![](https://i.imgur.com/XDW8KwH.png)|We can see that the first layer of the model has discovered weights representing edges (diagonal, horizontal, vertical) in addition to gradients.<br><br>These are foundational building blocks, similar to how basic visual machinery in the human eye!
2|![](https://i.imgur.com/b0TLzV9.png)|![](https://i.imgur.com/PTPric9.png)|We see that the model has learned to create feature detectors to see corners, repeating lines, circles, and other simple patterns.<br><br>Each weight picture matches small patches from example training data; e.g., sunsets on the bottom RHS.<br><br>These are built up from the foundational building blocks in layer 1.
3|![](https://i.imgur.com/EuDQvnS.png)|![](https://i.imgur.com/DBBrHxK.png)|We see the features learned are now able to identify and match with higher-level semantic components (e.g., printed text or even people)
4|![](https://i.imgur.com/gRRyrC2.png)|![](https://i.imgur.com/EuVmz5d.png)|We see that the model is now able to understand higher-level concepts, such as similar-looking dogs.
5|![](https://i.imgur.com/sYsWVEe.png)|![](https://i.imgur.com/ekbE8iq.png)|We see that the model can now identify different types of dogs, but classify them together!
</details>

In [ ]:
# Base pretrained model to piggy back off (refer to table above)
pre_trained_model = resnet34 

# How many times we will iterate through the each image in the training dataset
num_epochs = 1

learner = cnn_learner(
    dataloader,         # <- What data has been loaded into memory to train from
    pre_trained_model,  # <- Which pre-trained model we want to piggy-back off
    metrics=accuracy    # <- Report back the % of correct predictions
)

print(f"* Training model with {num_epochs} epoch...")

# Re-train the very last layers of the pre_trained_model for a specific purpose
learner.fine_tune(num_epochs)

print(f"* Done! Our model is {learner.metrics[0].value:.3%} accurate")

## 4️⃣: What did the model correctly and incorrectly learn?

FastAI has a [**`ClassificationInterpretation`**](https://docs.fast.ai/interpret.html#ClassificationInterpretation) class to help us understand what our model learnt correctly vs. incorrectly.


In [ ]:
learning_interpreter = ClassificationInterpretation.from_learner(learner)

# A confusion matrix will show us correct predicted vs. incorrect predictions
learning_interpreter.plot_confusion_matrix()

# We can plot the "top losses" - the higher the loss the lower the model is
# confident in the result
learning_interpreter.plot_top_losses(9, figsize=(15,10))

## 5️⃣: Running a prediction

1. Update `image_url` to any random accessable image on the internet
1. The cell to see which of the labels the model thinks it is

In [ ]:
import tempfile

# Grab some random images from the internet, and see what our model thinks it is
images = [
  "https://i.imgur.com/vqtjuLU.png",
  "https://i.imgur.com/Rzj3r7P.jpg",
  "https://i.imgur.com/wChxwgY.jpg",
]

for image_url in images: 
  image_path = tempfile.mktemp()

  print(f"* Downloading {image_url} to {image_path}...")
  ! wget -q --show-progress -O "$image_path" "$image_url"

  print(f"* Running prediction for this image...")
  _, _, predictions = learner.predict(image_path)

  print("* This how confident the model is for each label:")
  for index, label in enumerate(dataloader.vocab):
    print(f"    {label}:\t{predictions[index]:.3%}")

## 6️⃣: Cleaning up bad training data

> <i> 💩 in means 💩 out </i>

Run the following cell below, and delete or reclassify any bad images

In [ ]:
from fastai.vision.widgets import ImageClassifierCleaner
import shutil

# We can use an ImageClassifierCleaner on our learner to prune away mistakes
# or bad data
dataset_cleaner = ImageClassifierCleaner(learner, max_n=100)
dataset_cleaner

When done, run the following cell below to delete the bad images and move the misclassified images. 

Then re-train the model by going back to Step 3️⃣.

In [ ]:
# Go through all those images in the cleaner that we marked for deletion 
# and remove the files from our /datasets/{label} directory
print("* Deleting bad images:")
for i in dataset_cleaner.delete():
  image = dataset_cleaner.fns[i]
  image.unlink()
  print(f"  Deleted {image}...")
  
# Go through all those images in the cleaner marked for reclassification
# and move them to the right category subdirectory
print("* Moving misclassified images:")
for i, label in dataset_cleaner.change():
  image = dataset_cleaner.fns[i]
  move_to = datasets_dir / label
  shutil.move(str(image), move_to)
  print(f"  Moved {image} into {move_to}")

print("Done! Retrain your model again and check if there is any improvement!")

## 7️⃣: Save model to file

We can save our model to a [**Pickle file**](https://pythonnumericalmethods.berkeley.edu/notebooks/chapter11.03-Pickle-Files.html) <img src="https://emoji.slack-edge.com/T027TU47K/pickle-rick/cbbfd96e6c843c11.png" width="32">, which serialises the model to text.

In [ ]:
learner.export(fname="/my_model.pkl")

And now we can load in our model somewhere else, e.g.:

In [ ]:
from fastai import load_learner

learner = load_learner('/my_model.pkl')
learner.predict('/path/to/some/image')

## 💥 Additional Funsies

* What's the _least_ amount of training images you can use to train a decent model?
* Does increasing/decreasing image quality make a difference?
* Increase the number of epochs. Does it make your model better?
* Modify the training vs. validation dataset split proportions and see the results
* Switch to a different pre-trained model and see the results
* Cleanse more of your dataset to remove bad examples

![](https://i.imgur.com/SGE1hLZ.jpg)